# 一．PyG简介（PyG 1.3.2）
1. PyG为用户提供通用的MessagePassing接口，以便对新的研究想法进行快速干净的原型制作。此外，几乎所有近期提出的邻域聚合函数都适用于此接口，其中包括PyG已经集成的方法。

## 1.torch_geometric.data
### a.图数据转换
<img src="pic/PYG1.png" width="350"/> 

In [15]:
import torch
from torch_geometric.data import Data
#无向图表达时，两个节点之间的一条边需要用两个tuple表示

#边表达1：
#每个list代表一条有向边，list第一元素是起始节点，第二个元素是终止节点
edges1 = [[0,1],[1,0],[1,2],[2,1]]
edge_index1 = torch.tensor(edges1, dtype = torch.long)#先转化为tensor

#边表达2：
#两个list，第一个list存储起始节点，第二个list存储终止节点
edges2 = [[0, 1, 1, 2], [1, 0, 2, 1]]
edge_index2 = torch.tensor(edges2, dtype = torch.long)

node_features = [[-1], [0], [1]]
x = torch.tensor(node_features, dtype = torch.float)

data1 = Data(x=x, edge_index = edge_index1.t().contiguous())#转化为PyG DATA对象
data2 = Data(x=x, edge_index = edge_index2)

print(data1)
print(data2)
print(data1.edge_index)
print(data2.edge_index)
print("两种边的表达方式最终会转化成同一种Ｄata数据结构")

Data(edge_index=[2, 4], x=[3, 1])
Data(edge_index=[2, 4], x=[3, 1])
tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])
tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])
两种边的表达方式最终会转化成同一种Ｄata数据结构


### b.图数据的属性查看

In [52]:
print("查看data属性:",data1.keys)

print("查看节点特征:",data1['x'])

for key, item in data1:
    print("{} found in data".format(key))
    
'edge_attr' in data1
 
print("查看节点数:",data1.num_nodes)

print("查看边数",data1.num_edges)

print("查看节点特征维度:", data1.num_node_features)

print("查看是否包含独立节点:", data1.contains_isolated_nodes())

print("查看是否包含节点自环:", data1.contains_self_loops())

print("查看是否为有向图:", data1.is_directed())

'''
data所有方法可以在以下链接查看:
https://pytorch-geometric.readthedocs.io/en/1.3.2/modules/data.html#torch_geometric.data.Data
'''

查看data属性: ['x', 'edge_index']
查看节点特征: tensor([[-1.],
        [ 0.],
        [ 1.]])
edge_index found in data
x found in data
查看节点数: 3
查看边数 4
查看节点特征维度: 1
查看是否包含独立节点: False
查看是否包含节点自环: False
查看是否为有向图: False


'\ndata所有方法可以在以下链接查看:\nhttps://pytorch-geometric.readthedocs.io/en/1.3.2/modules/data.html#torch_geometric.data.Data\n'

### 2.torch_geometric.datasets(公共基准数据集)
<img src="pic/PYG2.png" width="450"/> 

### a.TUDataset

In [53]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root = "/home/jerry/local_git/notebook/Pytorch_Tutorail/PyG_Benchmark/ENZYMES", 
                    name = 'ENZYMES')

print("图数据个数:",len(dataset))
print("数据种类:",dataset.num_classes)
print("节点特征数:",dataset.num_node_features)

data = dataset[0]
print("第一张图数据信息:",data,"37个节点，每个节点3维，168/2=84条边，一个图标签")
#图数据分割
train_dataset = dataset[:540]
test_dataset = dataset[540:]
print("训练图数据规模:",train_dataset)
print("测试图数据规模:",test_dataset)
print("打乱数据集dataset.shuffle():",dataset.shuffle())

图数据个数: 600
数据种类: 6
节点特征数: 3
第一张图数据信息: Data(edge_index=[2, 168], x=[37, 3], y=[1]) 37个节点，每个节点3维，168/2=84条边，一个图标签
训练图数据规模: ENZYMES(540)
测试图数据规模: ENZYMES(60)
打乱数据集dataset.shuffle(): ENZYMES(600)


### b.Citeseer

In [55]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root="/home/jerry/local_git/notebook/Pytorch_Tutorail/PyG_Benchmark/Citeseer", 
                    name = "Citeseer")

print("图数据个数:", len(dataset))
print("数据种类:",dataset.num_classes)
print("节点特征维度:",dataset.num_node_features)
data = dataset[0]
print("图属性:",data)
print("data.train_mask:",data.train_mask)
print("data.train_mask个数:",data.train_mask.sum().item())

图数据个数: 1
数据种类: 6
节点特征维度: 3703
图属性: Data(edge_index=[2, 9104], test_mask=[3327], train_mask=[3327], val_mask=[3327], x=[3327, 3703], y=[3327])
data.train_mask: tensor([ True,  True,  True,  ..., False, False, False])
data.train_mask个数: 120


### 3.Mini-batches
### a.torch_geometric.data.DataLoader
<img src="pic/PYG3.png" width="450"/> 

In [69]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = dataset = TUDataset(root = "/home/jerry/local_git/notebook/Pytorch_Tutorail/PyG_Benchmark/ENZYMES", 
                              name = 'ENZYMES',
                              use_node_attr=True)
loader = DataLoader(dataset, batch_size = 32, shuffle=True)
#loader = DataLoader(dataset, batch_size = 32)

for batch in loader:
    print("一个batch属性:",batch)
    print("一个batch包含的图数据量:",batch.num_graphs)

print(batch.batch)

一个batch属性: Batch(batch=[1042], edge_index=[2, 4048], x=[1042, 21], y=[32])
一个batch包含的图数据量: 32
一个batch属性: Batch(batch=[988], edge_index=[2, 3914], x=[988, 21], y=[32])
一个batch包含的图数据量: 32
一个batch属性: Batch(batch=[893], edge_index=[2, 3536], x=[893, 21], y=[32])
一个batch包含的图数据量: 32
一个batch属性: Batch(batch=[907], edge_index=[2, 3550], x=[907, 21], y=[32])
一个batch包含的图数据量: 32
一个batch属性: Batch(batch=[1079], edge_index=[2, 4142], x=[1079, 21], y=[32])
一个batch包含的图数据量: 32
一个batch属性: Batch(batch=[928], edge_index=[2, 3668], x=[928, 21], y=[32])
一个batch包含的图数据量: 32
一个batch属性: Batch(batch=[1036], edge_index=[2, 3948], x=[1036, 21], y=[32])
一个batch包含的图数据量: 32
一个batch属性: Batch(batch=[1230], edge_index=[2, 4568], x=[1230, 21], y=[32])
一个batch包含的图数据量: 32
一个batch属性: Batch(batch=[1125], edge_index=[2, 4306], x=[1125, 21], y=[32])
一个batch包含的图数据量: 32
一个batch属性: Batch(batch=[1070], edge_index=[2, 4130], x=[1070, 21], y=[32])
一个batch包含的图数据量: 32
一个batch属性: Batch(batch=[1096], edge_index=[2, 3910], x=[1096, 21], y

### b.batch的含义：
<img src="pic/PYG4.png" width="800"/> 

### c.torch_scatter.scatter_mean
<img src="pic/PYG5.png" width="800"/> 
关于PyTorch Scatter的用法:
https://pytorch-scatter.readthedocs.io/en/latest/

In [76]:
from torch_scatter import scatter_mean
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = dataset = TUDataset(root = "/home/jerry/local_git/notebook/Pytorch_Tutorail/PyG_Benchmark/ENZYMES", 
                              name = 'ENZYMES',
                              use_node_attr=True)
loader = DataLoader(dataset, batch_size = 32, shuffle=True)

for data in loader:
    data
#在一个batch中获取每一个图的节点特征矩平均向量，组成一个矩阵
x = scatter_mean(data.x, data.batch, dim=0)
print(x.size())                           

torch.Size([24, 21])


### 4.Data Transforms(构建自己的数据集)
<img src="pic/PYG6.png" width="800"/> 
将17000个3D点云图转化为2D图

In [100]:
import urllib.request
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
response = urllib.request.urlopen('https://www.python.org')

from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/home/jerry/local_git/notebook/Pytorch_Tutorail/PyG_Benchmark/ShapeNet',
                   categories=['Airplane'])

print("3D point 数据：", dataset[0])

/home/jerry/anaconda3/lib/python3.8/site-packages/torch_geometric/data/dataset.py:89: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you really want to make use of another pre-processing technique, make sure to delete `/home/jerry/local_git/notebook/Pytorch_Tutorail/PyG_Benchmark/ShapeNet/processed/processed` first.
  warnings.warn(


3D point 数据： Data(category=[1], edge_index=[2, 15108], pos=[2518, 3], y=[2518])


<img src="pic/PYG7.png" width="800"/> 
使用KNN算法将3D点云图转化为2D图
# 3D图转化为2D图需要第一在下载数据集时就处理，如果是数据集已经处理过，则3D图转2D图失败

In [99]:
import urllib.request
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
response = urllib.request.urlopen('https://www.python.org')

# 使用KNN算法将3D云图转化为2D图
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/home/jerry/local_git/notebook/Pytorch_Tutorail/PyG_Benchmark/ShapeNet',
                   categories=['Airplane'],
                  pre_transform = T.KNNGraph(k=6))

print("3D point 数据 2D 化：", dataset[0])

3D point 数据 2D 化： Data(category=[1], edge_index=[2, 15108], pos=[2518, 3], y=[2518])


<img src="pic/PYG8.png" width="800"/> 
使用KNN算法将3D点云图转化为2D图，添加随机绕动
# 3D图转化为2D图需要第一在下载数据集时就处理，如果是数据集已经处理过，则3D图转2D图失败

In [98]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/home/jerry/local_git/notebook/Pytorch_Tutorail/PyG_Benchmark/ShapeNet',
                   categories=['Airplane'],
                  pre_transform = T.KNNGraph(k=6),
                  transform=T.RandomTranslate(0.01))

print("3D point 数据 2D 化,并随机化处理节点：", dataset[0])

3D point 数据 2D 化,并随机化处理节点： Data(category=[1], edge_index=[2, 15108], pos=[2518, 3], y=[2518])


### 5.GCN构建

In [101]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root="/home/jerry/local_git/notebook/Pytorch_Tutorail/PyG_Benchmark/Citeseer", 
                    name = "Citeseer")
print(dataset)

Citeseer()


In [107]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
        
        
    def forward(self, data):
        x, edge_index = data.x , data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(),
                             lr=0.01, 
                             weight_decay=5e-4)
# 转换为训练模式
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask],data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# 转换为测试模式
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))
    

Accuracy: 0.6760


# 二．构建消息传递网络--针对空间域图卷积网络的设计范式(Message Passing Networks)
![deque](pic/PYG9.png)

### a.计算无向图中节点的度
1. 图拓扑结构如下：

<img src="pic/PYG10.png" width="500"/> 

2. degree方法:

<img src="pic/PYG11.png" width="500"/> 

3. add_self_loop方法：

<img src="pic/PYG12.png" width="500"/> 

4. torch.nn.Linear()方法：

<img src="pic/PYG13.png" width="500"/> 

In [10]:
import torch 
from torch_geometric.data import Data
#无向图表达时，两个节点之间的一条边需要用两个tuple表示

#边表达2：
#两个list，第一个list存储源始节点，第二个list存储目标节点
edges = [[0, 0, 0, 1, 2, 2, 3, 3], [1, 2, 3, 0, 0, 3, 0, 2]]
edge_index = torch.tensor(edges, dtype = torch.long)

node_features = [[-1,1,2], [1,1,1], [0,1,2], [3,1,2]]
x = torch.tensor(node_features, dtype = torch.float)

data = Data(x=x, edge_index = edge_index)

print("原始数节点数:",data.num_nodes)
print("原始数边数:",data.num_edges)

print("原始数据:",data)

from torch_geometric.utils import add_self_loops,degree

#给图数据的边增加自环边
self_loop_edge, _= add_self_loops(data.edge_index, num_nodes=data.num_nodes)

print("增加了自环边数据:", self_loop_edge)

edge_index_j, edge_index_i = self_loop_edge

print("edge_index_j:",edge_index_j)
print("edge_index_i:",edge_index_i)

print("计算增加自环图数据中起始节点边关系列表的度：", degree(edge_index_j))
print("计算原始图数据起始节点边关系列表的度：",degree(data.edge_index[0],
                            num_nodes=data.num_nodes))

#对节点度进行运算操作：

edge_index_j_, edge_index_i_ = data.edge_index

# 计算图数据起始节点边关系列表的度
deg = degree(edge_index_j_)

# 计算１/sqrt(d_{i})
deg_inv_sqrt = deg.pow(-0.5)

print("计算图数据起始节点边关系列表的度:deg:",deg)
print("计算１/sqrt(d_{i}):",deg_inv_sqrt)

print("edge_index_j_:", edge_index_j_)
print("edge_index_i_:", edge_index_i_)

# GCN 邻居节点聚合权重计算
GCN_weight = deg_inv_sqrt[ edge_index_j_]*deg_inv_sqrt[ edge_index_i_]
print("deg_inv_sqrt[ edge_index_j_]:",deg_inv_sqrt[ edge_index_j_])
print("deg_inv_sqrt[ edge_index_i_]:",deg_inv_sqrt[ edge_index_i_])
print("GCN_weight:",GCN_weight)
print("将行向量GCN_weight变为列向量GCN_weight.view(-1,1):",GCN_weight.view(-1,1))

# 构建带学习参数的线性变换
input_features = 3 # 节点的特征维度
output_features = 5
linear_transform = torch.nn.Linear(input_features, output_features)

x_t = linear_transform(data.x)

print("x_t.shape:",x_t.shape)

print("线性变换后的node_features_matrix:",x_t)

# 构造有向图
s = [1,2,3,2]
t = [0,0,0,3]
edges = [s,t]
edge_index = torch.tensor(edges, dtype = torch.long)

node_features = [[-1,1,2], [1,1,1], [0,1,2], [3,1,2]]
x = torch.tensor(node_features, dtype = torch.float)
data = Data(x=x, edge_index = edge_index)
print("有向图edge_index表示:\n",data.edge_index)
# 有向图增加环
self_edge_index,_ = add_self_loops(data.edge_index, num_nodes=data.num_nodes)
print("增加自环的有向图edge_index表示:\n",self_edge_index)
print("判断data是否时有向图(增加自环操作不会改变data本身的节点边连接关系):\n",data.is_directed())

原始数节点数: 4
原始数边数: 8
原始数据: Data(edge_index=[2, 8], x=[4, 3])
增加了自环边数据: tensor([[0, 0, 0, 1, 2, 2, 3, 3, 0, 1, 2, 3],
        [1, 2, 3, 0, 0, 3, 0, 2, 0, 1, 2, 3]])
edge_index_j: tensor([0, 0, 0, 1, 2, 2, 3, 3, 0, 1, 2, 3])
edge_index_i: tensor([1, 2, 3, 0, 0, 3, 0, 2, 0, 1, 2, 3])
计算增加自环图数据中起始节点边关系列表的度： tensor([4., 2., 3., 3.])
计算原始图数据起始节点边关系列表的度： tensor([3., 1., 2., 2.])
计算图数据起始节点边关系列表的度:deg: tensor([3., 1., 2., 2.])
计算１/sqrt(d_{i}): tensor([0.5774, 1.0000, 0.7071, 0.7071])
edge_index_j_: tensor([0, 0, 0, 1, 2, 2, 3, 3])
edge_index_i_: tensor([1, 2, 3, 0, 0, 3, 0, 2])
deg_inv_sqrt[ edge_index_j_]: tensor([0.5774, 0.5774, 0.5774, 1.0000, 0.7071, 0.7071, 0.7071, 0.7071])
deg_inv_sqrt[ edge_index_i_]: tensor([1.0000, 0.7071, 0.7071, 0.5774, 0.5774, 0.7071, 0.5774, 0.7071])
GCN_weight: tensor([0.5774, 0.4082, 0.4082, 0.5774, 0.4082, 0.5000, 0.4082, 0.5000])
将行向量GCN_weight变为列向量GCN_weight.view(-1,1): tensor([[0.5774],
        [0.4082],
        [0.4082],
        [0.5774],
        [0.4082],
   

### b.使用消息传递网络构建GCN单层网络前向传播过程
### MessagePassing类中函数自动调用逻辑链：

self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)——>

message(self, x_j, edge_index, size)——>

update(self, aggr_out)

1.　输入的拓扑图结构:
<img src="pic/PYG10.png" width="500"/> 
2. 输入x_features_matrix的维度自动转换： 

2.1. 经过线性变换后的x,shape=[4, 5]
<img src="pic/PYG14_1.png" width="1500"/> 
<img src="pic/PYG14_2.png" width="1000"/> 
2.2. 调用self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)——>message(self, x_j, edge_index, size)

x_j自动将x从shape[N, out_channels],N表示途中节点数量，对应edge_index起始节点列表节点信息转换为shape[E,out_channels ]，Ｅ表示edge_index中list的长度.

2.3.x_j shape:
<img src="pic/PYG16.png" width="500"/> 
2.4.edge_index:
<img src="pic/PYG15.png" width="1000"/> 
2.5.norm.view(-1,1)*x_j:
<img src="pic/PYG17.png" width="800"/> 
## 3.update(self, aggr_out)||super(GCNConv, self).__init__(aggr='add')

## PYG按照edge_index_i list(目标节点边索引列表, target node edge index list)对以edge_index_j list(起始节点边索引列表 source node edge index list)扩展的node feature matrix(使用目标节点边索引扩展的节点特征矩阵)进行聚合,得到本次聚合后的节点特征矩阵X.

## norm.view(-1,1)*x_j的node_features_matrix基于edge_index_i中目标节点列表indx选择对应norm.view(-1,1)中行向量使用(aggr='add')中选择的add方式进行消息聚合
### 3.1.edge_index(蓝色点表示０号节作为目标节点在edge_index中的索引):
<img src="pic/PYG18.png" width="800"/> 
### 3.2.result(蓝色点表示1号节点，2号节点，3号节点0号节作为起始节点对目标节点0在result_features_matrix中的特征向量):
<img src="pic/PYG19.png" width="800"/> 
### 3.3.aggr_out蓝色点表示0号节经过add聚合后的特征向量
<img src="pic/PYG20.png" width="800"/> 
### 3.4.edge_index_j表示起始节点索引列表
### 3.5.edge_index_i表示目标节点索引列表
### 3.6.x_j表示x以edge_index_j列表索引扩展的特征矩阵  
### 3.7.x_i表示x以edge_index_i列表索引扩展的特征矩阵
<img src="pic/PYG29.png" width="800"/> 
<img src="pic/PYG10.png" width="500"/> 
4. GCN算法：
<img src="pic/PYG24.png" width="800"/> 

In [17]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.utils import add_self_loops, degree

class GCN(MessagePassing):
    def __init__(self, in_channels, out_channels, normalize_embedding=True):
        super(GCN, self).__init__(aggr="add")
        # 预定义线性变换操作输入输出维度
        self.linear_transfomer = torch.nn.Linear(in_channels, out_channels)
        
        if normalize_embedding:
            self.normalize_emb = True

    def forward(self, x, edge_index):
        # 给图增加自环
        edge_index,_ = add_self_loops(edge_index, num_nodes=x.size(0))
        # 特征线性变换
        x = self.linear_transfomer(x)
        # 消息传递,将图信息传递给message模块进行加工
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        edge_index_j, edge_index_i = edge_index
        # edge_index_j起始节点边关系列表
        # edge_index_i目标节点边关系列表
        deg = degree(edge_index_j)
        # 求edge_index_j起始节点边关系列表度开根号的倒数.
        deg_inv_sqrt = deg.pow(-0.5)
        
        GCN_weight = deg_inv_sqrt[edge_index_j]*deg_inv_sqrt[edge_index_i]
        # 按　edge_index_j　list 中索引号在　deg_inv_sqrt　list　取值构成新　list
        
        # GCN_weight.view(-1,1)行向量转列向量
        GCN_weight = GCN_weight.view(-1,1)
        #图中起始节点特征矩阵乘以对应weight后的节点特征表示矩阵，为信息聚合做准备
        weight_node = GCN_weight*x_j
        """
        edge_index_j表示初始节点索引列表
        x_j表示x以edge_index_j列表索引扩展的特征矩阵
        """
        return weight_node

    def update(self, aggr_out, x):
        # step 4
        add_aggr = aggr_out
        # aggr_out 是选择super(GraphGCN, self).__init__(aggr="add")定义的"add"方法聚合weight_node特征向量.
        # step 5

        return add_aggr

# data
# 构图
#edges = [[0, 0, 0, 1, 2, 2, 3, 3], [1, 2, 3, 0, 0, 3, 0, 2]]

edges = [[0, 0, 0, 1, 2, 2, 3, 3], [1, 2, 3, 0, 0, 3, 0, 2]]
edge_index = torch.tensor(edges, dtype=torch.long)

node_features = [[-1, 1, 2], [1, 1, 1], [0, 1, 2], [3, 1, 2]]
x = torch.tensor(node_features, dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

x = data.x
edge_index = data.edge_index

GraphGCN = GCN(3, 5)

x = GraphGCN(x, edge_index)
print(x)

tensor([[ 0.5985,  0.4058, -1.1607, -0.3281,  0.9678],
        [ 0.4292,  0.2020, -0.8265, -0.4245,  0.5796],
        [ 0.4491,  0.3468, -1.0333, -0.2742,  0.9022],
        [ 0.4491,  0.3468, -1.0333, -0.2742,  0.9022]],
       grad_fn=<ScatterAddBackward>)


# 三、GraphSAGE 单层网络前向计算构建 
1. GraghSAGE算法原理:
<img src="pic/PYG23.png" width="800"/> 
2. torch.cat:
<img src="pic/PYG21.png" width="800"/> 
<img src="pic/PYG22.png" width="500"/> 
3. F.normalize(aggr_out,p=2,dim=1)
<img src="pic/PYG25.png" width="800"/> 

In [20]:
#GraphSage
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
import torch.nn.functional as F

class GraphSage(MessagePassing):
    def __init__(self, in_channels, out_channels, normalize_embedding=True):
        super(GraphSage, self).__init__(aggr="mean")
         # 预定义线性变换操作输入输出维度
        self.aggr_lin = torch.nn.Linear(2*in_channels, out_channels)
        if normalize_embedding:
            self.normalize_emb = True

    def forward(self, x, edge_index):
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        return x_j

    def update(self, aggr_out, x):
        # step 4
        mean_aggr = aggr_out
        # aggr_out 是选择super(GraphGCN, self).__init__(aggr="mean)定义的"mean"方法聚合后的特征向量.
        
        # step 5
        # 特征拼接（特征增强）
        concat_out = torch.cat((x, mean_aggr), 1)  # x, aggr_out shape相同，按行合并
        
        # self.aggr_lin(concat_out)降维， F.relu()非线性变换
        aggr_out = F.relu(self.aggr_lin(concat_out))
        # step 7
        if self.normalize_emb:
            aggr_out = F.normalize(aggr_out, p=2, dim=1)
            # p = 2　２范数
            # dim = 1,表示此操作是横向进行
            # dim =　0,表示此操作时列向进行
        return aggr_out


# data
# 构图
edges = [[0, 0, 0, 1, 2, 2, 3, 3], [1, 2, 3, 0, 0, 3, 0, 2]]
edge_index = torch.tensor(edges, dtype=torch.long)

node_features = [[-1, 1, 2], [1, 1, 1], [0, 1, 2], [3, 1, 2]]
x = torch.tensor(node_features, dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

x = data.x
edge_index = data.edge_index

conv = GraphSage(3, 5)
x = conv(x, edge_index)
print(x)

tensor([[0.2119, 0.0156, 0.8516, 0.2007, 0.4351],
        [0.0346, 0.3485, 0.9075, 0.0000, 0.2321],
        [0.0344, 0.1906, 0.8564, 0.0449, 0.4766],
        [0.0000, 0.4588, 0.8087, 0.0000, 0.3681]], grad_fn=<DivBackward0>)


# 四、GAT 单层网络单头注意力机制前向计算构建 
1. nn.Parameter()：
<img src="pic/PYG26.png" width="800"/> 
2. GAT注意力机制在PYG中的实现：
<img src="pic/PYG27.png" width="800"/> 
3. 单头GAT公式：
<img src="pic/PYG28.png" width="800"/> 

In [22]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.utils import add_self_loops, remove_self_loops,softmax

class GAT(MessagePassing):

    def __init__(self, in_channels, out_channels, dropout=0, bias=True,):
        super(GAT, self).__init__(aggr='add')
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.dropout = dropout
        self.lin = torch.nn.Linear(in_channels, self.out_channels)
        self.att = torch.nn.Parameter(torch.Tensor(1, self.out_channels*2))
        #因为节点特征需要拼接后再乘以节点特征注意力权重向量a,所以节点特征注意力权重向量a的维度为self.out_channel*2

        if bias:
            self.bias = torch.nn.Parameter(torch.Tensor(self.out_channels))
        else:
            self.register_parameter('bias', None)

        torch.nn.init.xavier_uniform_(self.att)
        torch.nn.init.zeros_(self.bias)


    def forward(self, x, edge_index, size=None):
        if size is None and torch.is_tensor(x):
            edge_index, _ = remove_self_loops(edge_index)
            edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        # 原始特征线性变换：
        x = self.lin(x)
        return self.propagate(edge_index, size=size, x=x)

    def message(self, edge_index_j,edge_index_i, x_i, x_j, size_i):

        # x_i = x_i.view(-1, 1, self.out_channels)
        # x_j = x_j.view(-1, 1, self.out_channels)

        alpha = (torch.cat([x_i, x_j], dim=-1) * self.att).sum(dim=-1)
        # sum(dim=-1): 按行进求和;
        # torch.cat([x_i, x_j], dim=-1): x_i矩阵与x_j矩阵按行拼接;
        # torch.cat([x_i, x_j], dim=-1) * self.att: self.att 注意力系数点乘x_i与x_j矩阵拼接后每一行;
        # (torch.cat([x_i, x_j], dim=-1) * self.att).sum(dim=-1): sum(dim=-1)对shape[12,12]按行求和

        alpha = F.leaky_relu(alpha, 0.2)
        # 求每行值的激活值

        alpha = softmax(alpha, edge_index_i, size_i)
        #alpha计算是起始节点到目标节点注意力权重，也是就节点edge_index_j——>节点edge_index_i的注意力权重

        alpha = F.dropout(alpha, p=self.dropout, training=self.training)#self.dropout = 1,没有进行dropout处理
        
        result = x_j * alpha.view(-1, 1)
        # x_j表示按edge_index_j扩展的初始节点特征矩阵

        return result

    def update(self, aggr_out):
        # aggr_out: result　以 edge_index_i 节点序号为聚合顺利,按aggr=add的方式进行聚合
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out
# data
edges = [[0, 0, 0, 1, 2, 2, 3, 3], [1, 2, 3, 0, 0, 3, 0, 2]]
edge_index = torch.tensor(edges, dtype=torch.long)

node_features = [[-1, 1, 2], [1, 1, 1], [0, 1, 2], [3, 1, 2]]
x = torch.tensor(node_features, dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

x = data.x
edge_index = data.edge_index


conv = GAT(3, 6)
x = conv(x, edge_index)

print(x)

tensor([[-0.3735,  0.7449,  0.0608, -0.4438,  1.4067, -1.0382],
        [-0.4167,  0.3876, -0.0779, -0.4843,  1.4095, -0.7658],
        [-0.4016,  0.7115,  0.0482, -0.5224,  1.5565, -0.9724],
        [-0.4016,  0.7115,  0.0482, -0.5224,  1.5565, -0.9724]],
       grad_fn=<AddBackward0>)


# 五、GAT多头注意力机制原理
## 核心思想：在单头gat上平行的使用多次独立注意力机制，其中每一次注意力机制的参数独立
1. 单层一个神经元神经网络：
<img src="pic/GAT5.png" width="800"/> 
2. GAT论文原理介绍（gat中注意力机制ａ使用的是单层一个神经元神经网络结构）:
<img src="pic/GAT1.png" width="800"/> 
<img src="pic/GAT2.png" width="800"/> 
<img src="pic/GAT3.png" width="800"/> 
<img src="pic/GAT4.png" width="800"/> 